Import required libraries and set up paths.
Download the CPHD file if needed.
Open the CPHD file and read metadata.
Extract a block of phase history data.
Form an intensity image using FFTs (range and azimuth).
Crop and scale the image for output.
Extract the SRP and set up the local CRS.
Write the image as a GeoTIFF.

Cell 2: Imports, path setup, and download logic.
Cell 4: Open CPHD and get metadata.
Cell 8: Extract a block of data and form the image (FFT processing).
Cell 9: Crop and scale the image to 8-bit.
Cell 10: Extract SRP, set up CRS, and write the GeoTIFF.

Cell 2: Imports, path setup, and download logic.

In [ ]:
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from sarpy.io.phase_history.cphd import CPHDReader
import rasterio
from rasterio.transform import from_origin
from pathlib import Path

plt.rcParams["figure.dpi"] = 120

# Where we keep big SAR data (mounted Windows share in WSL)
DATA_DIR = "/mnt/bigbasement/Projects/SATSAR"

# Capella CPHD we’ll use for this demo
CPHD_S3 = (
    "s3://capella-open-data/data/2025/8/26/"
    "CAPELLA_C13_SP_CPHD_HH_20250826023518_20250826023527/"
    "CAPELLA_C13_SP_CPHD_HH_20250826023518_20250826023527.cphd"
)
CPHD_LOCAL = f"{DATA_DIR}/CAPELLA_C13_SP_CPHD_HH_20250826023518_20250826023527.cphd"

# Check and download if needed
if not os.path.exists(CPHD_LOCAL):
    print("CPHD file not found locally. Downloading from S3...")
    subprocess.run([
        "aws", "s3", "cp", "--no-sign-request",
        CPHD_S3,
        CPHD_LOCAL
    ], check=True)
else:
    print("CPHD file already exists locally. Skipping download.")

# Display for sharing (only filename, not full path)
print("Data directory:", Path(DATA_DIR).name)
print("CPHD file:", Path(CPHD_LOCAL).name)


Cell 4: Open CPHD and get metadata.

In [ ]:
reader = CPHDReader(CPHD_LOCAL)
meta = reader.cphd_meta

print("✅ CPHD opened")

print("\n=== CollectionID ===")
print(meta.CollectionID)

print("\n=== Global (frequency band, timing) ===")
print(meta.Global)

print("\n=== Antenna (gain, phase center) ===")
print(meta.Antenna)

print("\n=== ReferenceGeometry (platform/scene geometry) ===")
print(meta.ReferenceGeometry)

print("\n=== Channel (polarization, bandwidth, identifiers) ===")
# Channel is a ChannelType (not a dict); print it directly:
print(meta.Channel)


Cell 8: Extract a block of data and form the image (FFT processing)

In [ ]:
# Choose a mid-swath region to form (you can tune these)
num_pulses  = 2048
start_pulse = 40000

# Pull a block: (azimuth, range)
pulse_block = reader[start_pulse:start_pulse + num_pulses, :]
print("pulse_block shape:", pulse_block.shape, "dtype:", pulse_block.dtype)

# Window to control sidelobes
num_range = pulse_block.shape[1]
win_range = np.hanning(num_range).astype(np.float32)
win_az    = np.hanning(num_pulses).astype(np.float32)
pulse_block_win = pulse_block * (win_az[:, None] * win_range[None, :])

# Range compression (FFT across range)
rng_fft = np.fft.fftshift(np.fft.fft(pulse_block_win, axis=1), axes=1)
# Azimuth compression (FFT across pulses)
img_fft = np.fft.fftshift(np.fft.fft(rng_fft, axis=0), axes=0)

# Log magnitude image
power = np.abs(img_fft)
power[power <= 0] = 1e-12
log_img = 20*np.log10(power)

# Display with robust contrast
vmin, vmax = np.percentile(log_img, [5, 99])
plt.figure(figsize=(6,6))
plt.imshow(log_img, cmap="gray", vmin=vmin, vmax=vmax, aspect="auto")
plt.title("Quicklook intensity (2D FFT approx)")
plt.xlabel("range (FFT domain)")
plt.ylabel("azimuth (FFT domain)")
plt.show()


Cell 9: Crop and scale the image to 8-bit.

In [ ]:
# Remove empty energy at FFT edges (heuristic 10% crop in range)
h, w = log_img.shape
left, right = int(w*0.10), int(w*0.90)
log_img_crop = log_img[:, left:right]
print("Cropped shape:", log_img_crop.shape)

# Normalize to 8-bit for a simple preview image
l, r = np.percentile(log_img_crop, [5, 99.5])
scaled = np.clip((log_img_crop - l) / (r - l), 0, 1)
preview_8bit = (scaled * 255).astype(np.uint8)

plt.figure(figsize=(6,6))
plt.imshow(preview_8bit, cmap="gray", aspect="auto")
plt.title("Cropped + scaled preview (our formed image)")
plt.xlabel("range px")
plt.ylabel("azimuth px")
plt.show()

Cell 10: Extract SRP, set up CRS, and write the GeoTIFF.

In [ ]:
# 7a) SRP ECEF from metadata you printed above
srp = meta.ReferenceGeometry.SRP.ECF
srp_x, srp_y, srp_z = float(srp.X), float(srp.Y), float(srp.Z)

# 7b) ECEF → lat/lon/alt (WGS84)
a  = 6378137.0
e2 = 6.69437999014e-3

def ecef_to_llh(x, y, z):
    lon = np.arctan2(y, x)
    p = np.sqrt(x*x + y*y)
    lat = np.arctan2(z, p*(1 - e2))
    for _ in range(5):
        N = a / np.sqrt(1 - e2*np.sin(lat)**2)
        alt = p/np.cos(lat) - N
        lat = np.arctan2(z, p*(1 - e2*(N/(N+alt))))
    N = a / np.sqrt(1 - e2*np.sin(lat)**2)
    alt = p/np.cos(lat) - N
    return lat, lon, alt

lat, lon, alt = ecef_to_llh(srp_x, srp_y, srp_z)
lat_deg, lon_deg = float(np.degrees(lat)), float(np.degrees(lon))
print("SRP lat/lon/alt:", lat_deg, lon_deg, alt)

# 7c) Local, meter-based CRS: Azimuthal Equidistant centered at SRP
local_crs = (
    f"+proj=aeqd +lat_0={lat_deg} +lon_0={lon_deg} "
    f"+x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"
)

# 7d) Pixel spacing assumptions (first-pass, defensible)
#   Range resolution ~ c/(2*BW); for BW=600 MHz → ~0.25 m.
#   Azimuth spacing is rough here (no full motion comp) → ~1.0 m.
xres = 0.25  # meters/px (range columns)
yres = 1.00  # meters/px (azimuth rows)

az_pixels, rg_pixels = preview_8bit.shape
xmin = - (rg_pixels * xres) / 2.0
ymax =   (az_pixels * yres) / 2.0
transform = from_origin(xmin, ymax, xres, yres)

# 7e) Write GeoTIFF
out_tif = f"{DATA_DIR}/capella_quicklook_proto.tif"
with rasterio.open(
    out_tif, "w",
    driver="GTiff",
    height=az_pixels,
    width=rg_pixels,
    count=1,
    dtype=preview_8bit.dtype,
    crs=local_crs,
    transform=transform,
) as dst:
    dst.write(preview_8bit, 1)

print("Wrote:", out_tif)